In [1]:
#import modules and set up environment 
import os
import sys
path = "../../src/"

sys.path.append(path)
import glob
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from tqdm import tqdm
import jax.numpy as jnp
from jax import jit, grad, vmap, hessian, scipy, random

#import sgmcmc code 
import models.logistic_regression.logistic_regression as lr
import samplers.sgd as sgd
import samplers.sgld as sgldcv
import samplers.sgldps as sgldcvps

from metrics import logloss

In [2]:
#load in the data
file_path = "../../data/real/covtype_train.csv"
test_path = "../../data/real/covtype_test.csv"
data = pd.read_csv(file_path)
dat_array = data.values[:]
N = dat_array.shape[0]
x = np.column_stack([np.ones(N), dat_array[:, 1:]])
y = dat_array[:,0]
test_data = pd.read_csv(test_path)
test_array = test_data.values[:]
N_test = test_array.shape[0]
x_test = np.column_stack([np.ones(N_test), test_array[:, 1:]])
y_test = test_array[:,0]

#set up model parameters
dim = x.shape[1] 

#priors
mu_0 = np.zeros(dim) #prior mean
lambda_0 = 10.0*np.eye(dim)  #prior covariance matrix

### Importing csv files

In [3]:
methods = ["sgld", "sgldps", "sgldcv", "sgldcvps"]
sgld_batches = 0.01
samples_csv = dict()
Niter_sgld = 10**4

for i in range(len(methods)):
    method = methods[i]
    items = glob.glob(f"./out/lrb_{method}_samples.csv")[0]
    samples_csv[method] = pd.read_csv(items).iloc[(Niter_sgld+1):].reset_index(drop=True)

### Log-loss calculation

In [6]:
Nrep = 10
y_test = y_test.reshape((N_test, 1))

for k in tqdm(range(len(methods)), desc= "Methods"):
    method = methods[k]
    idx=np.arange(Niter_sgld, 0, -10)[::-1]-1
    ll_arr = np.zeros((idx.shape[0], Nrep))
    for j in range(Nrep):
        for i in range(idx.shape[0]):
            index = idx[i]
            sample = samples_csv[method].values[index, (dim*(j+1)):dim*(j+2)]
            ll_arr[i,j] = logloss(sample, x_test, y_test)
            
        
    ll_arr_df = pd.DataFrame(ll_arr)
    ll_arr_df.to_csv(f"./out/llcover_{method}.csv", index=False)

Methods: 100%|██████████| 4/4 [11:37<00:00, 174.35s/it]
